# Unidad 03

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

## Ejercicios Subconsultas
1. Crear un reporte que muestre el nombre del producto y el id del proveedor para todos los productos suministrados por “Exotic Liquids”, “Grandma Kelly's Homestead”, y “Tokyo Traders”. (NombreEmpresa)


In [ ]:
SELECT P.NombreProducto, P.IDProveedor
FROM Productos P 
WHERE P.IDProveedor IN 
    (SELECT PR.IDProveedor
    FROM Proveedores PR
    WHERE PR.NombreEmpresa IN (
    'Exotic Liquids', 
    'Grandma Kelly''s Homestead', 
    'Tokyo Traders')
);


2. Crear un reporte que muestre todos los productos por nombre que son de la categoría “Mariscos”

In [ ]:
SELECT P.NombreProducto
FROM Productos P
WHERE P.IDCategoria IN 
    (SELECT C.IDCategoria
    FROM Categorias C
    WHERE C.NombreCategoria = 'Mariscos');

3. Crear un reporte que muestre todas las compañias por su nombre, que comercialicen productos de la categoría 8. 

In [ ]:
SELECT PR.NombreEmpresa
FROM Proveedores PR
WHERE PR.IDProveedor IN 
    (SELECT P.IDProveedor
    FROM Productos P
    WHERE P.IDCategoria = 8);


4. Crear un reporte que muestre por su nombre a todas las compañías que comercializan productos de la categoría “Mariscos”

In [ ]:
SELECT PR.nombreempresa
FROM Proveedores PR
WHERE PR.IDProveedor IN 
    (SELECT P.IDProveedor
    FROM Productos P
    WHERE P.IDCategoria IN 
        (SELECT C.IDCategoria
        FROM Categorias C
        WHERE C.NombreCategoria = 'Mariscos'));


## Expresiones de Tabla

In [ ]:
SELECT *
FROM (SELECT IDCliente, NombreEmpresa
FROM Clientes
WHERE Pais = 'EE.UU.') AS USAClies;

In [ ]:
SELECT YEAR(FechaPedido) AS ordenanio, COUNT(DISTINCT IDCliente) AS numclies
FROM Pedidos
GROUP BY ordenanio;

In [ ]:
SELECT ordenanio, 
COUNT(DISTINCT IDCliente) AS numclies
FROM (SELECT YEAR(FechaPedido) AS ordenanio, IDCliente
FROM Pedidos) AS D
GROUP BY ordenanio;

In [ ]:
SELECT ordenanio, 
COUNT(DISTINCT IDCliente) AS numclies
FROM (SELECT YEAR(FechaPedido), IDCliente
FROM Pedidos) AS D(ordenanio, IDCliente)
GROUP BY ordenanio;

In [ ]:
SELECT ordenanio, numcusts
FROM (SELECT ordenanio, COUNT(DISTINCT IDCliente) AS numcusts
FROM (SELECT YEAR(FechaPedido) AS ordenanio, IDCliente
FROM Pedidos) AS D1
GROUP BY ordenanio) AS D2
WHERE numcusts > 70;

| Año  | Clientes Año Actual | Clientes Año Anterior | Crecimiento |
|------|---------------------|-----------------------|-------------|
| 2019 | 67                  |                       |             |
| 2020 | 86                  | 67                    | 19          |
| 2021 | 81                  | 86                    | -5          |

In [ ]:
SELECT Cur.ordenanio AS Año, Cur.numcusts AS ClientesAñoActual, Prv.numclies AS ClientesAñoAnterior, Cur.numcusts - Prv.numclies AS Crecimiento
FROM (SELECT YEAR(FechaPedido) AS ordenanio, COUNT(DISTINCT IDCliente) AS numcusts
FROM Pedidos GROUP BY YEAR(FechaPedido)) AS Cur
LEFT OUTER JOIN
(SELECT YEAR(FechaPedido) AS ordenanio, COUNT(DISTINCT IDCliente) AS numclies
FROM Pedidos GROUP BY YEAR(FechaPedido)) AS Prv
ON Cur.ordenanio = Prv.ordenanio + 1;

In [ ]:
WITH UsaClies AS
(
SELECT IDCliente, NombreEmpresa
FROM Clientes
WHERE Pais = 'EE.UU.'
)
SELECT * FROM UsaClies;

In [ ]:
WITH C AS
(
SELECT YEAR(FechaPedido) AS ordenanio, IDCliente
FROM Pedidos
)
SELECT ordenanio, COUNT(DISTINCT IDCliente) AS numcusts
FROM C
GROUP BY ordenanio;

In [ ]:
WITH C(ordenanio, IDCliente) AS
(
SELECT YEAR(FechaPedido), IDCliente
FROM Pedidos
)
SELECT ordenanio, COUNT(DISTINCT IDCliente) AS numcusts
FROM C
GROUP BY ordenanio;

In [ ]:
WITH C1 AS
(
SELECT YEAR(FechaPedido) AS ordenanio, IDCliente
FROM Pedidos
),
C2 AS
(
SELECT ordenanio, COUNT(DISTINCT IDCliente) AS numclies
FROM C1
GROUP BY ordenanio
)
SELECT ordenanio, numclies
FROM C2
WHERE numclies > 70;

| Año  | Clientes Año Actual | Clientes Año Anterior | Crecimiento |
|------|---------------------|-----------------------|-------------|
| 2019 | 67                  |                       |             |
| 2020 | 86                  | 67                    | 19          |
| 2021 | 81                  | 86                    | -5          |

In [ ]:
WITH CuentaAnual AS
(
SELECT YEAR(FechaPedido) AS ordenanio,
COUNT(DISTINCT IDCliente) AS numclies
FROM Pedidos
GROUP BY YEAR(FechaPedido)
)
SELECT Cur.ordenanio AS Año,
Cur.numclies AS ClientesAñoActual, Prv.numclies AS ClientesAñoAnterior,
Cur.numclies - Prv.numclies AS Crecimiento
FROM CuentaAnual AS Cur
LEFT OUTER JOIN CuentaAnual AS Prv
ON Cur.ordenanio = Prv.ordenanio + 1;

In [ ]:
WITH FactorialCTE (Numero, Factorial) AS
(
    SELECT 1 Numero, 1 Factorial
    UNION ALL
    SELECT Numero + 1, Factorial * (Numero + 1)
    FROM FactorialCTE
    WHERE Numero < 10
)
SELECT Numero, Factorial
FROM FactorialCTE
OPTION (MAXRECURSION 0);


In [ ]:
--Consulta Recursiva para listar al empleado y todos los subordinados en el organigrama hasta el mas bajo en la piramide
WITH EmpCTE (IDEmpleado, Nombre, Apellido, puesto, jefeid)
AS
(
SELECT IDEmpleado, Nombre, Apellido, Puesto, JefeID
FROM Empleados
WHERE IDEmpleado = 5
UNION ALL
--RECURSIVIDAD
SELECT e.IDEmpleado, e.Nombre, e.Apellido, e.Puesto, e.JefeID
FROM Empleados AS e JOIN EmpCTE AS m
ON e.JefeID = m.IDEmpleado
)
-- Desplegar ResultSet
SELECT * FROM EmpCTE;

In [ ]:
--Consulta Recursiva para listar al empleado y todos los subordinados en el organigrama hasta el mas bajo en la piramide
WITH EmpCTE (IDEmpleado, Nombre, Apellido, puesto, jefeid)
AS
(
SELECT IDEmpleado, Nombre, Apellido, Puesto, JefeID
FROM Empleados
WHERE IDEmpleado = 5
UNION ALL
--RECURSIVIDAD
SELECT e.IDEmpleado, e.Nombre, e.Apellido, e.Puesto, e.JefeID
FROM Empleados AS e JOIN EmpCTE AS m
ON e.IDEmpleado = m.jefeid
)
-- Desplegar ResultSet
SELECT * FROM EmpCTE;

In [ ]:
DROP VIEW IF EXISTS ClientesUSA;
GO
CREATE VIEW ClientesUSA
AS
SELECT
IDCliente, NombreEmpresa, NombreContacto, PuestoContacto, Direccion,
Ciudad, Region, CodigoPostal, Pais, Telefono, Fax
FROM Clientes
WHERE Pais = 'EE.UU.';
GO

In [ ]:
SELECT IDCliente, NombreEmpresa, Pais FROM ClientesUSA;

In [ ]:
DROP FUNCTION IF EXISTS GetClieOrdenes;
GO
CREATE FUNCTION GetClieOrdenes
(@cid AS char(5)) RETURNS TABLE
AS
RETURN
SELECT IDPedido, IDCliente, IDEmpleado, FechaPedido, FechaRequerida,
FechaEnvio, EnvioPor, Flete, NombreEnvio, DireccionEnvio, CiudadEnvio,
RegionEnvio, CodigoPostalEnvio, PaisEnvio
FROM Pedidos
WHERE IDCliente = @cid;
GO

In [ ]:
SELECT IDPedido, IDCliente
FROM GetClieOrdenes('ALFKI') AS O;

In [ ]:
SELECT O.IDPedido, O.IDCliente, DP.IDProducto, DP.Cantidad
FROM GetClieOrdenes('ALFKI') AS O
JOIN [Detalles Pedido] AS DP
ON O.IDPedido = DP.IDPedido;